In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt
import seaborn as sns
sns.set_theme()

def read_and_plot(file_path, title, scopy_file=None):
    df = pd.read_csv(file_path, delimiter="\t", skiprows=1, encoding="ISO-8859-1", names=["Freq", "Magnitude"])
    df[['Magnitude_dB', 'Phase_deg']] = df['Magnitude'].str.extract(r'\((-?[\d\.e\+]+)dB,(-?[\d\.e\+]+)°\)')
    df['Magnitude_dB'] = pd.to_numeric(df['Magnitude_dB'])
    df['Phase_deg'] = pd.to_numeric(df['Phase_deg'])
    df['Freq'] = pd.to_numeric(df['Freq'])
    df = df.dropna(subset=['Magnitude_dB', 'Phase_deg', 'Freq'])


    fig, ax = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

    # LTSpice plots
    ax[0].semilogx(df['Freq'], df['Magnitude_dB'], label='LTSpice Gain (dB)', color='tab:green')
    ax[1].semilogx(df['Freq'], df['Phase_deg'], label='LTSpice Phase (deg)', color='tab:green')

    if scopy_file:
        df_scopy = pd.read_csv(scopy_file, skiprows=7, encoding="ISO-8859-1", delimiter=",")
        df_scopy['Frequency(Hz)'] = pd.to_numeric(df_scopy['Frequency(Hz)'])
        df_scopy['Magnitude(dB)'] = pd.to_numeric(df_scopy['Magnitude(dB)'])
        df_scopy['Phase(°)'] = pd.to_numeric(df_scopy['Phase(°)'])


        scopy_mag_interp = np.interp(df['Freq'], df_scopy['Frequency(Hz)'], df_scopy['Magnitude(dB)'])
        scopy_phase_interp = np.interp(df['Freq'], df_scopy['Frequency(Hz)'], df_scopy['Phase(°)'])

        ax[0].semilogx(df['Freq'], scopy_mag_interp, label='Scopy Measured Gain', color='tab:blue', linestyle='dashed')
        ax[1].semilogx(df['Freq'], scopy_phase_interp, label='Scopy Measured Phase', color='tab:red', linestyle='dashed')

    ax[0].set_ylabel("Magnitude (dB)")
    ax[1].set_ylabel("Phase (deg)")
    ax[1].set_xlabel("Frequency (Hz)")

    ax[0].legend()
    ax[1].legend()
    ax[0].grid(True, which="both", linestyle="--")
    ax[1].grid(True, which="both", linestyle="--")

    fig.suptitle(title)
    fig.tight_layout()
    plt.show()

    return df
file_paths = {
    'C:\\home\\ece2001\\reports\\report6\\circuit1.txt': {
        'title': 'Circuit 1, RC High-Pass Filter',
        'scopy': 'C:\\home\\ece2001\\reports\\report6\\circuit1.csv'
    },
    'C:\\home\\ece2001\\reports\\report6\\circuit2.txt': {
        'title': 'Circuit 2, RC Low-Pass Filter',
        'scopy': 'C:\\home\\ece2001\\reports\\report6\\circuit2.csv'
    },
    'C:\\home\\ece2001\\reports\\report6\\circuit3.txt': {
        'title': 'Circuit 3, RC Parallel',
        'scopy': 'C:\\home\\ece2001\\reports\\report6\\circuit3.csv'
    },
    'C:\\home\\ece2001\\reports\\report6\\circuit4a.txt': {
        'title': 'Circuit 4a, R=200',
        'scopy': 'C:\\home\\ece2001\\reports\\report6\\circuit4.csv'
    },
    'C:\\home\\ece2001\\reports\\report6\\circuit4b.txt': {
        'title': 'Circuit 4b, R=2000',
        #'scopy': '/home/art/school/ece2001/reports/report6/scopy4b.csv'
    },
}

dfs = {}
for idx, (path, info) in enumerate(file_paths.items()):
    dfs[idx] = read_and_plot(path, info['title'], scopy_file=info.get('scopy'))

circuit1 = dfs[0]
circuit2 = dfs[1]
circuit3 = dfs[2]
circuit4a = dfs[3]
circuit4b = dfs[4]


print(max(circuit1['Magnitude_dB']))
print(max(circuit2['Magnitude_dB']))
print(max(circuit3['Magnitude_dB']))
print(max(circuit4a['Magnitude_dB']))
print(max(circuit4b['Magnitude_dB']))



2.60534964646415
2.60534802730252
2.60564237566306
10.0740792801379
13.9782157017905
